In [10]:
import os
import shutil
import json
import skimage
import numpy as np
import pandas as pd
from collections import OrderedDict

# 원본 annotation 파일들이 있는 경로
annoResampleDir = "/home/boo/Desktop/dataset/Training/label/"

# anno_dir 내 annotation 파일 이름 리스트업
annoResampleList = os.listdir(annoResampleDir)
print(len(annoResampleList))
print(annoResampleList[:10])

64801
['22_111229_220623_25.json', '08_150721_220615_03.json', '08_150721_220615_32.json', '22_100653_221020_25.json', '08_105027_220627_10.json', '08_144931_220613_18.json', '08_143736_221012_40.json', '21_104728_220620_25.json', '08_193143_220608_04.json', '08_193837_220628_25.json']


In [11]:
anno_dict_resample = {"pedestrian":0,"trafficLight":0,"trafficSign":0, "twoWheeler":0, "vehicle":0}

In [13]:
# 안전보호구 5클래스(1,2,5,7,8)만 남기고 나머지 다 삭제하기
# 수정 후 json 파일 변경
# .DS_Store 파일이 간혹 생길때가 있습니다. 폴더 내에 해당 파일이 존재하면 오류가 발생합니다. 반드시 찾아서 삭제 후 진행하세요.
from tqdm import tqdm
f = open("/home/boo/Desktop/dataset/Training/resampling.txt", 'w')

# remove_list = ["03","04","06","09","10","11","12","13","14","15","16","17","18","19","20","21","22","23","24","25",
#             "26","27","28","29","30","31","32","33","34","35","36","37","38","39","40","41","42","43","44","45"]

for anno in tqdm(annoResampleList):
    try :
        with open(annoResampleDir + anno, encoding="utf-8-sig", errors="ignore") as json_file:
            jsonData = json.load(json_file)
    except : # .DS_Store가 있는지 확인하기 위해 작성, 일반적으로 mac Finder에는 해당 파일이 보이지 않으나 SFTP 시스템 등으로 보면 해당 파일이 있으니 삭제합니다.
        print(anno)
    imgPath = anno.replace(".json", ".jpg")
    df = pd.json_normalize(jsonData)
    filename = df["information.filename"][0]
    annoList = df["annotations"][0]

    
#     print(filename)
#    for i in annoList[:]:
#        if i["class"] in remove_list:
#            annoList.remove(i)
    
    if annoList : 
        for j in range(len(annoList)) : 
            item = annoList[j]["class"]
            # class imblance를 조정하기 위해서 현재는 Tao yolo에서 class_weight를 지원하지만, 애초에 전처리 과정에서 개수를 제한해 적당한 비율 맞추기
            # 해당 레이블 개수가 10000개를 이하이면
            if anno_dict_resample[item] < 30000 :
                anno_dict_resample[item] += 1
                # 레이블이 10000개를 넘기지 않을 때만 이미지명을 저장한다. 중복이 생기지만 추후에 제거한다.
                f.write(filename+'\n')
        
    with open(annoResampleDir + anno, 'w', encoding="utf-8-sig", errors="ignore") as outfile:
        json.dump(jsonData, outfile)
        
f.close()

100%|██████████| 64801/64801 [00:49<00:00, 1320.60it/s]


In [14]:
# coding:utf-8
import os
import shutil
import json
import skimage
import numpy as np
import pandas as pd
from collections import OrderedDict

# 중복을 제거하고 남은 이미지에서 다시 한번 클래스의 분포를 확인한다
annoResampleDir = "/home/boo/Desktop/dataset/Training/resampling_label/"
# anno_dir 내 annotation 파일 이름 리스트업
annoResampleList = os.listdir(annoResampleDir)
print(len(annoResampleList))
print(annoResampleList[:10])

anno_dict_resample = {"pedestrian":0,"trafficLight":0,"trafficSign":0, "twoWheeler":0, "vehicle":0}

for anno in annoResampleList:
    with open(annoResampleDir + anno, encoding="utf-8-sig", errors="ignore") as json_file:
        jsonData = json.load(json_file)
    imgPath = anno.replace(".json", ".jpg")
    df = pd.json_normalize(jsonData)
    filename = df["information.filename"][0]
    #print(df["image.filename"][0])
    for i in range(len(df["annotations"][0])):
        item = df["annotations"][0][i]["class"] 
        anno_dict_resample[item] += 1
        
# 값(value)를 기준으로 정렬한 OrderedDict
ordered_dict = OrderedDict(sorted(anno_dict_resample.items(), key=lambda t:t[1], reverse=True))
print(ordered_dict)

23661
['22_111229_220623_25.json', '08_150721_220615_03.json', '08_150721_220615_32.json', '22_100653_221020_25.json', '08_105027_220627_10.json', '08_144931_220613_18.json', '08_143736_221012_40.json', '21_104728_220620_25.json', '08_193143_220608_04.json', '08_193837_220628_25.json']
OrderedDict([('vehicle', 165503), ('trafficSign', 53436), ('trafficLight', 47559), ('pedestrian', 35775), ('twoWheeler', 20000)])
